In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,
    return_messages=True,
)

example_message_prompt = ChatPromptTemplate.from_messages(
    [        
        ("human", "Express it with exactly 3 emoticons that symbolize the movie {movie}."),          
        ("ai", "{emogi}"),
    ]
)

examples = [
    {
        "movie": "Top Gun",
        "emogi": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "emogi": "👨‍👨‍👦🔫🍝"
        
    },
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_message_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-class movie critic."),
        MessagesPlaceholder(variable_name="history"),
        few_shot_prompt,
        ("human", "{question}"), 
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [43]:
print(load_memory(_))

[]


In [44]:
invoke_chain("Squid Game")

content='🔴🎪💰'


In [45]:
print(load_memory(_))

[HumanMessage(content='Squid Game'), AIMessage(content='🔴🎪💰')]


In [46]:
invoke_chain("old boy")

content='👦🔨📦'


In [47]:
print(load_memory(_))

[HumanMessage(content='Squid Game'), AIMessage(content='🔴🎪💰'), HumanMessage(content='old boy'), AIMessage(content='👦🔨📦')]


In [48]:
invoke_chain("tell me what are the movies I asked about before. Show me the emoticon you created, too.")

content='1. Squid Game: 🔴🎪💰\n2. Old Boy: 👦🔨📦\n3. Top Gun: 🛩️👨\u200d✈️🔥\n4. The Godfather: 👨\u200d👨\u200d👦🔫🍝'


In [49]:
print(load_memory(_))

[SystemMessage(content='The human mentions "Squid Game" and the AI responds with emojis. The human then brings up "Old Boy" and the AI responds with emojis related to the movie.'), HumanMessage(content='tell me what are the movies I asked about before. Show me the emoticon you created, too.'), AIMessage(content='1. Squid Game: 🔴🎪💰\n2. Old Boy: 👦🔨📦\n3. Top Gun: 🛩️👨\u200d✈️🔥\n4. The Godfather: 👨\u200d👨\u200d👦🔫🍝')]


In [42]:
memory.clear()